In [7]:
import datetime
import ujson
import numpy 
data = ujson.load(open('user_project_times.json'))

In [8]:
summaries = {}
ctr = 0
for line in open('user_project_summaries.json'): # lazy iteration because the file is large
    print ctr,
    ctr+=1
    summaries.update(ujson.loads(line))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46


Create Language Data Set and Initialize Language Codes

In [9]:
'''We are using all the data from the user from whom we we have previously 
classfied what their primary langauge is '''
user_langs = ujson.load(open('user_inferredlangs.json')) 
lang_data = {}
keys = user_langs.keys()

#remove Latin 
# noLatin = []
# for key in keys: 
#     if "la" != user_langs[key:
#         noLatin.append(key)
# keys = noLatin
                          
print 'done loading keys'
for key in keys: 
    lang_data[key] = data[key[0:2] + '/' + key]
print 'lang data created'

lang_dict = {} #key is langauge isocode, value is number 
isocodes = ujson.load(open('isocodes.json'))
counter = 0 
for lang in isocodes:
    lang_dict[lang] = counter
    counter+=1
    

done loading keys
lang data created


In [10]:
def convert_time(el):
    el = int(str(el)[:10])
    return datetime.date.fromtimestamp(el)

In [11]:
earliest = 1e100
latest = 0
for user in data:
    for ctime, _ in data[user]: #ignore last-modified for now
        if ctime < earliest:
            earliest = ctime
        if ctime > latest:
            latest = ctime

print 'Earliest:', convert_time(earliest) 
print 'Latest:', convert_time(latest)
MAXDUR = 300
MINDUR = 150
# get subset of data from "old" users whose earliest creation date is more than MAXDUR days before the end of the dataset
# (giving them a good chance to stay on for more than MINDUR days),
# AND have activity within MAXDUR days if they are active beyond MINDUR (to be fair to recent users)
old_data = {}
for user in data:
    start_time = min([ctime for ctime, _ in data[user]])
    if int((latest-start_time)/(86400.*1000))>MAXDUR:
        post = filter(lambda length: length>MINDUR, 
                      map(lambda (ctime,mtime): (ctime-start_time)/(86400.*1000), data[user]))
        if len(post)>0 and len(filter(lambda length: length<=MAXDUR, post))==0:
            continue  # ignore users who have no activity in the 120-180 period
        old_data[user[3:]] = summaries[user[3:]]
print 'Filtered to', len(old_data), 'users from', len(data)

old_user_activity_length = numpy.zeros(len(old_data))
for i, user in enumerate(old_data.keys()):
    ctimes = [ctime for ctime, _ in data[user[0:2] + '/' + user]]
    old_user_activity_length[i] = (max(ctimes) - min(ctimes))/(86400*1000)

Earliest: 2013-03-27
Latest: 2016-03-10
Filtered to 25324 users from 46319


In [12]:
TASK = "RETENTION"
if TASK == 'RETENTION': 
    data = old_data
if TASK == 'LANGUAGE':
    data = lang_data

In [13]:
import sys 
import os 
from features import * 
sys.path.append(os.getcwd())


In [14]:
def getEarliest(userID): #helper function to get the earliest and latest project creation date for summmaries
    user = summaries[userID]
    earliest = user.keys()[0]
    for proj in user: 
        creationDate = summaries[userID][proj]['**created']
        if  creationDate < earliest: 
            earliest = creationDate
    return earliest
        

In [15]:
def getProjectsInMINDUR(userID): #for summaries
    projDict = {}
    earliest = getEarliest(userID)

    latest = (MINDUR * 86400 * 1000) + earliest 
    user = summaries[userID]
    for proj in user: 
        if summaries[userID][proj]['**created'] >= earliest and summaries[userID][proj]['**created'] <= latest: 
            projDict[proj] = summaries[userID][proj]
    return projDict
                        
                                                                                 

In [16]:
def getData(userID):
    if TASK == 'LANGUAGE': 
        return {userID: summaries[userID]} 
    if TASK == 'RETENTION':
        return {userID : getProjectsInMINDUR(userID)}

In [17]:
def getProjects(userID): 
    r = getData(userID)
    projects = r[r.keys()[0]] 
    #print projects[projects.keys()[0]]['**created']

    name_time = [(project, projects[project]['**created']) for project in projects]
    name_time = sorted(name_time, key=lambda t: t[1])  
    proj_list = [summaries[userID][project[0]] for project in name_time]
    return proj_list
    #print proj_list


In [18]:
def percentProjectsFeatures(projects):
    """return a dictionary mapping each time slot (10%, 20%, etc) to the percentage of projects created
    at that time,
    for a particular user represented as list of (creation times, modified times),
    using only their first MINDUR days of activity"""   
    userDict = {} 
    #note: feature names are for our reference only
    userDict["average length"] = getAverageProjectLen(projects)
    userDict["num projects"] = numProjects(projects)
    
    decileProjects = numProjectsInDecile(projects)
    
    userDict["1st"] = decileProjects[0]
    userDict["2nd"] = decileProjects[1]
    userDict["3rd"] = decileProjects[2]
    userDict["4th"] = decileProjects[3]
    userDict["5th"] = decileProjects[4]
    userDict["6th"] = decileProjects[5]
    userDict["7th"] = decileProjects[6]
    userDict["8th"] = decileProjects[7]
    userDict["9th"] = decileProjects[8]
    userDict["10th"] = decileProjects[9]

    return userDict


In [19]:
def dayAnalysisFeatures(projects): 
    userDict = {} 
    userDict["weekday"] = percentOnWeekday(projects) #
    day = percentOnDay(projects)
    userDict["Monday"] = day[0]
    userDict["Tuesday"] = day[1]
    userDict["Wednesday"] =day[2]
    userDict["Thursday"] = day[3]
    userDict["Friday"] = day[4]
    userDict["Saturday"] = day[5]
    userDict["Sunday"] = day[6]

    return userDict

In [20]:
def summaryOBlockDecile(projects): 
    userDict = {}
    decileOrphanBlock = decileOrphanBlocks(projects)
    userDict["O 1"] = decileOrphanBlock[0]
    userDict["O 2"] = decileOrphanBlock[1]
    userDict["O 3"] = decileOrphanBlock[2]
    userDict["O 4"] = decileOrphanBlock[3]
    userDict["O 5"] = decileOrphanBlock[4]
    userDict["O 6"] = decileOrphanBlock[5]
    userDict["O 7"] = decileOrphanBlock[6]
    userDict["O 8"] = decileOrphanBlock[7]
    userDict["O 9"] = decileOrphanBlock[8]
    userDict["O 10"] = decileOrphanBlock[9]
    return userDict

In [21]:
def summaryDecileTLBlocks(projects): 
    userDict = {}
    decileTypesTopLevelBlocks = decileNumTopLevelBlocks(projects)
    userDict["TL 1"] = decileNumTopLevelBlock[0]
    userDict["TL 2"] = decileNumTopLevelBlock[1]
    userDict["TL 3"] = decileNumTopLevelBlock[2]
    userDict["TL 4"] = decileNumTopLevelBlock[3]
    userDict["TL 5"] = decileNumTopLevelBlock[4]
    userDict["TL 6"] = decileNumTopLevelBlock[5]
    userDict["TL 7"] = decileNumTopLevelBlock[6]
    userDict["TL 8"] = decileNumTopLevelBlock[7]
    userDict["TL 9"] = decileNumTopLevelBlock[8]
    userDict["TL 10"] = decileNumTopLevelBlock[9]
    return userDict

In [22]:
def summaryDecileNumScreens(projects): 
    userDict = {}
    decileNumS = decileNumScreens(projects)
    userDict["NS 1"] = decileNumS[0]
    userDict["NS 2"] = decileNumS[1]
    userDict["NS 3"] = decileNumS[2]
    userDict["NS 4"] = decileNumS[3]
    userDict["NS 5"] = decileNumS[4]
    userDict["NS 6"] = decileNumS[5]
    userDict["NS 7"] = decileNumS[6]
    userDict["NS 8"] = decileNumS[7]
    userDict["NS 9"] = decileNumS[8]
    userDict["NS 10"] = decileNumS[9]
    return userDict

In [23]:
def summaryAverages(projects):
    userDict = {} 
    userDict["NS"] = averageNumScreens(projects)
    userDict["NB"] = averageNumBlocks(projects)
    userDict["OB"] = getAverageOrphanBlocks(projects)
    userDict["TL"] = getAverageTypeTLBlocks(projects)
    userDict["TL2"] = getAverageNumTLBlocks(projects)
    userDict["NC"] = averageNumComponents(projects)
    userDict["MC"] = aveNumMediaAssets(projects)
    varList = getAllVariables(projects)
    
    userDict["local vars"] = varList[0]
    userDict["global vars"] = varList[1]
    
    return userDict

Write other feature functions -- perhaps different interval sizes (instead of 10%), proportion of projects rather than number, etc. With some effort, we can get a measure of activity on weekends/evenings as features too. Experiment...

Give each feature function a distinct name, and re-run the above two cells with the function in place of `featurize` to see if the contingency tables change

Remember from the CS111 assignment that we can combine a list of feature functions to give a new feature function, so you should experiment with combinations as well.

In [24]:
def combine_featfuncs(funclist):
    def combined(user):
        basedict = funclist[0](user)
        for f in funclist[1:]:
            basedict.update(f(user))
        return basedict
    return combined

RETENTION:

In [31]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer

user_dicts = []
all_features = []
time_features = [] 
code_features = []

# i = 0

if TASK == "RETENTION":
    i = 0
    for user in data:
        i +=1 
        if i % 1000 == 0: 
            print i
        projects = getProjects(user)
        all_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages])
        time_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures])
        code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) 
    
        all_features.append(all_combined(projects))
        time_features.append(time_combined(projects))
        code_features.append(code_combined(projects))


    print ('done loading featurizers')
    

    vec = DictVectorizer()
    X1 = vec.fit_transform(all_features) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html
    X2 = vec.fit_transform(time_features)
    X3 = vec.fit_transform(code_features)
    #X = vec.fit_transform(user_dicts) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html

    #X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 



ImportError: cannot import name __check_build

**Attempt 1: Clustering.** Is there any correlation between the clusters produced by these features, and whether or not a user is retained?

In [32]:
from sklearn.cluster import KMeans
if TASK == 'RETENTION':

    KMeans_model = KMeans(n_clusters=2)
    users_clusterids = KMeans_model.fit_predict(X1)

    from scipy.stats import chi2_contingency
    conting = numpy.zeros((2, 2))  # row = cluster ID (0, 1), col = #days (0 for <=120, 1 for >120)

    for i in range(len(users_clusterids)):
        cluster = users_clusterids[i]
        if old_user_activity_length[i]<=MINDUR:
            conting[cluster, 0] += 1
        else:
            conting[cluster, 1] += 1

# visualize
    print conting

# compute chi2 statistic to get the probability that there is no association between
# the clustering and whether or not the user is active for >120 days
# lower probabilities are better
    _, p, _, _ = chi2_contingency(conting)
    print 'There is a', p, 'probability that the clusters and retention are independent.'

ImportError: cannot import name __check_build

**Attempt 2: Classification.**

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn import linear_model

if TASK=='RETENTION':
    y = numpy.array(map(lambda d: int(d>MINDUR), old_user_activity_length))  # labels (0 for less than MINDUR, 1 otherwise)
    #X1 = normalize(X1, axis=0, norm='l1')
    X2 = normalize(X2, axis=0, norm='l1')
    percentpos = sum(y)/float(len(y))
    chance = max(percentpos, 1-percentpos)
    print 'Chance is', chance

    print 'Building a model with', X1.shape[1], 'features for', TASK
    all_coefs = [] 
    for trainidx, testidx in StratifiedKFold(y):
        print 'Fold'
        k = 70 #num neighbors
        ytrain = y[trainidx]
        ytest = y[testidx]
        
        X1train = X1[trainidx, :]  # using numpy's smart indexing
        X1test = X1[testidx, :]
        
        X2train = X2[trainidx, :]  # using numpy's smart indexing
        X2test = X2[testidx, :]
    
        X3train = X3[trainidx, :]  # using numpy's smart indexing
        X3test = X3[testidx, :]
    
        KNeighbors_model1 = KNeighborsClassifier(n_neighbors=k)
        logistic_model1 = linear_model.LogisticRegression()

        KNeighbors_model2 = KNeighborsClassifier(n_neighbors=k)
        logistic_model2 = linear_model.LogisticRegression()

        KNeighbors_model3 = KNeighborsClassifier(n_neighbors=k)  
        logistic_model3 = linear_model.LogisticRegression()

        KNeighbors_model1.fit(X1train, ytrain)
        logistic_model1.fit(X1train, ytrain)

        KNN_XtestP1 = KNeighbors_model1.predict(X1test)
        log_XtestP1 = logistic_model1.predict(X1test)
    
        KNeighbors_model2.fit(X2train, ytrain)
        logistic_model2.fit(X2train, ytrain)

        KNN_XtestP2 = KNeighbors_model2.predict(X2test)
        log_XtestP2 = logistic_model2.predict(X2test)
        
        KNeighbors_model3.fit(X3train, ytrain)
        logistic_model3.fit(X3train, ytrain)

        KNN_XtestP3 = KNeighbors_model3.predict(X3test)
        log_XtestP3 = logistic_model3.predict(X3test)
    
    
    
        print k, 'neighbors' 
        print "All features KNN Score: ", KNeighbors_model1.score(X1test, ytest)
        print 'All features KNN f1 score', f1_score(KNN_XtestP1, ytest),'\n'
        
        print'All features Logistic Regression Score: ', logistic_model1.score(X1test, ytest)
        print 'All features Logistic f1 score', f1_score(log_XtestP1, ytest) 
        print "Number predicted who survived", float(sum(log_XtestP1))/ len(log_XtestP1)


        print "Time features KNN Score: ", KNeighbors_model2.score(X2test, ytest)
        print 'Time features KNN f1 score', f1_score(KNN_XtestP2, ytest),'\n'

        print'Time features Logistic Regression Score: ', logistic_model2.score(X2test, ytest)
        print 'Time features Logistic f1 score', f1_score(log_XtestP2, ytest) 
        
        print "Code features KNN Score: ", KNeighbors_model3.score(X3test, ytest)
        print 'Code features KNN f1 score', f1_score(KNN_XtestP3, ytest),'\n'

        print'Code features Logistic Regression Score: ', logistic_model3.score(X3test, ytest)
        print 'Code features Logistic f1 score', f1_score(log_XtestP3, ytest) 
        
        


LANGUAGE:
    

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer
from features import *
all_features = []
time_features = [] 
code_features = []
if TASK == "LANGUAGE": 
    users = lang_data.keys()
    print ('done loading keys')
    i = 0 
    for user in users:
        i +=1 
        if i % 1000 == 0: 
            print i
        projects = getProjects(user)
        all_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages])
        time_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures])
        code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) 

        all_features.append(all_combined(projects))
        time_features.append(time_combined(projects))
        code_features.append(code_combined(projects))

    # for userID in lang_data: #user is just the user's id num 
    #     combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, decileFeatures ,summaryNumScreensDecile,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) #a function object is being returned here?? 
    #     user_dicts.append(combined(userID))


    print ('done loading featurizers')


    vec = DictVectorizer()
    X1 = vec.fit_transform(all_features) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html
    X2 = vec.fit_transform(time_features)
    X3 = vec.fit_transform(code_features)
    #X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 



In [ ]:
# print len(data)
# print len(lang_data)
# #what y is
# print [lang_dict[user_langs[userID]] for userID in lang_data]

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn.metrics import confusion_matrix


print old_user_activity_length[0:100]
if TASK=='LANGUAGE':
    y =  numpy.array([lang_dict[user_langs[userID]] for userID in lang_data]) # labels

    percentpos = sum(y)/float(len(y))
    chance = max(percentpos, 1-percentpos)
    print 'Chance is', chance

    #print 'Building a model with', X1.shape[1], 'features for', TASK
    all_coefs = [] 
    for trainidx, testidx in StratifiedKFold(y):
        print 'Fold'
        k = 70 #num neighbors
        X1train = X1[trainidx, :]  # using numpy's smart indexing
        X1test = X1[testidx, :]
        ytrain = y[trainidx]
        ytest = y[testidx]

        X2train = X2[trainidx, :]  # using numpy's smart indexing
        X2test = X2[testidx, :]

        X3train = X3[trainidx, :]  # using numpy's smart indexing
        X3test = X3[testidx, :]

        KNeighbors_model1 = KNeighborsClassifier(n_neighbors=k)  
        KNeighbors_model2 = KNeighborsClassifier(n_neighbors=k)  
        KNeighbors_model3 = KNeighborsClassifier(n_neighbors=k)  

        print('models created')

        KNeighbors_model1.fit(X1train, ytrain)
        print 'model1 fit'
        KNeighbors_model2.fit(X2train, ytrain)
        print 'model2 fit'
        KNeighbors_model3.fit(X3train, ytrain)  
        print ('all models fit')

        KNN_XtestP1 = KNeighbors_model1.predict(X1test)
        print 'model1 predicted'
        KNN_XtestP2 = KNeighbors_model2.predict(X2test)
        print 'model2 predicted'
        KNN_XtestP3 = KNeighbors_model3.predict(X3test)
        print 'models predicted \n'

        print k, 'neighbors' 
        print "All features KNN Score: ", KNeighbors_model1.score(X1test, ytest)
        print 'All features KNN f1 score', f1_score(KNN_XtestP1, ytest),'\n'

        print "time features KNN Score: ", KNeighbors_model2.score(X2test, ytest)
        print 'time features KNN f1 score', f1_score(KNN_XtestP2, ytest),'\n'

        print "code features KNN Score: ", KNeighbors_model3.score(X3test, ytest)
        print 'code features KNN f1 score', f1_score(KNN_XtestP3, ytest),'\n'

    cnf_matrix = confusion_matrix(ytest, KNN_XtestP1)



In [ ]:
cnf_matrix = list(cnf_matrix)
print len(cnf_matrix)
print len(lang_dict)